In [1]:
from langchain_community.document_loaders import PyPDFLoader

# 문서 로드
file_path = "../example_data/KCI_FI003153549.pdf"
loader = PyPDFLoader(file_path)

pages = loader.load()

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # 최대 길이(1000자 이하)
    chunk_overlap=200 # 정확히 200자 중첩
)

chunks = splitter.split_documents(pages)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(
        model="models/gemini-embedding-001",
        google_api_key=gemini_api_key
    )

In [4]:
lst = [chunk.page_content for chunk in chunks]

In [6]:
embeddings = embeddings_model.embed_documents(lst)

In [7]:
len(embeddings)

42

In [ ]:
embeddings[0]

#### FAISS 벡터스토어 저장

In [6]:
from langchain_community.vectorstores import FAISS

In [5]:
vector_store = FAISS.from_documents(chunks, embeddings_model) # 메모리에 저장(in-memory)

NameError: name 'FAISS' is not defined

In [ ]:
vector_store.save_local("./faiss_index") # 영구적으로 디스크에 저장(local disk)

In [7]:
vector_store = FAISS.load_local(
        "./faiss_index", 
        embeddings_model, 
        allow_dangerous_deserialization=True
    )

In [8]:
vector_store

In [9]:
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"

#### similarity_search() 메서드

In [11]:
result = vector_store.similarity_search(query, k=3)

In [12]:
result

[Document(id='c264a709-b6da-420c-9777-1ca8e1284784', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-27T02:09:00+09:00', 'moddate': '2024-12-27T02:09:00+09:00', 'source': '../example_data/KCI_FI003153549.pdf', 'total_pages': 12, 'page': 4, 'page_label': '5'}, page_content='의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하\n기 위해 의료기기 임상시험 전문가로부터 총 158개의 문\n서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음\n과 같이 분류된다:\n\x9f 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, \nGCP 문서 등\n\x9f 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라\n인 강의 자료 등\n\x9f 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR \n(Clinical Study Report) 템플릿 등\n\x9f 의료기기 특화 문서 (15%): 의료기기 임상시험 계획\n서, 기술문서 등\n\x9f 기타 (10%): 윤리위원회 관련 문서, 환자 동의서 템플\n릿 등\n1.2 Validity of Collected Data\n수집된 데이터셋은 의료기기 임상시험에 특화된 \nPrivate LLM 구축을 위해 도메인 적합성과 다양성, 그리\n고 응용 가능성 측면에서 높은 타당성을 갖추고 있다. 총 \n111,954페이지로 구성된 데이터는 의료기기 임상시험의 \n규제, 프로토콜 설계, 데이터 관리 등 전반적인 지식을 포\n괄하며, 국제 표준과 실제 임상시험 환경에서 발생할 수 \n있는 다양한 시나리오를 반영하도록 설계되었다.\n수집된 데이터는 규제 문서(30%), 교육 자료(20%), 프\n로토콜 및 보고서(25

In [10]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    '''다음 컨텍스트만 사용해 질문에 답하세요.
컨텍스트:{context}

질문:{question}
'''
)

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                            google_api_key=gemini_api_key)

In [12]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 컨텍스트만 사용해 질문에 답하세요.\n컨텍스트:{context}\n\n질문:{question}\n')

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [18]:
chain = prompt | llm | output_parser

In [22]:
resonse = chain.invoke({"context": result, "question": query})

In [24]:
print(resonse)

본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**이며, 문서 유형별 비율은 다음과 같습니다:

*   **규제 문서:** 30%
*   **교육 자료:** 20%
*   **프로토콜 및 보고서:** 25%
*   **의료기기 특화 문서:** 15%
*   **기타:** 10%


#### as_retriever() 메서드

In [14]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [15]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000224F0309160>, search_kwargs={'k': 3})

In [16]:
retriever.invoke(query)

[Document(id='c264a709-b6da-420c-9777-1ca8e1284784', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-27T02:09:00+09:00', 'moddate': '2024-12-27T02:09:00+09:00', 'source': '../example_data/KCI_FI003153549.pdf', 'total_pages': 12, 'page': 4, 'page_label': '5'}, page_content='의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하\n기 위해 의료기기 임상시험 전문가로부터 총 158개의 문\n서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음\n과 같이 분류된다:\n\x9f 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, \nGCP 문서 등\n\x9f 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라\n인 강의 자료 등\n\x9f 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR \n(Clinical Study Report) 템플릿 등\n\x9f 의료기기 특화 문서 (15%): 의료기기 임상시험 계획\n서, 기술문서 등\n\x9f 기타 (10%): 윤리위원회 관련 문서, 환자 동의서 템플\n릿 등\n1.2 Validity of Collected Data\n수집된 데이터셋은 의료기기 임상시험에 특화된 \nPrivate LLM 구축을 위해 도메인 적합성과 다양성, 그리\n고 응용 가능성 측면에서 높은 타당성을 갖추고 있다. 총 \n111,954페이지로 구성된 데이터는 의료기기 임상시험의 \n규제, 프로토콜 설계, 데이터 관리 등 전반적인 지식을 포\n괄하며, 국제 표준과 실제 임상시험 환경에서 발생할 수 \n있는 다양한 시나리오를 반영하도록 설계되었다.\n수집된 데이터는 규제 문서(30%), 교육 자료(20%), 프\n로토콜 및 보고서(25

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | output_parser
)

In [ ]:
response = chain.invoke(query) # query는 RunnablePassthrough()를 통과하여 question이라는 키의 값이 됨

In [19]:
print(response)

본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**입니다.

문서 유형별 비율은 다음과 같습니다:
*   **규제 문서**: 30%
*   **교육 자료**: 20%
*   **프로토콜 및 보고서**: 25%
*   **의료기기 특화 문서**: 15%
*   **기타**: 10%
